In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

dataset = load_diabetes()

X = pd.DataFrame(data=dataset['data'], columns=dataset['feature_names'])

X = StandardScaler().fit_transform(X)

y= pd.DataFrame(dataset['target'], columns=['progression'])

In [2]:
X

array([[ 0.80050009,  1.06548848,  1.29708846, ..., -0.05449919,
         0.41855058, -0.37098854],
       [-0.03956713, -0.93853666, -1.08218016, ..., -0.83030083,
        -1.43655059, -1.93847913],
       [ 1.79330681,  1.06548848,  0.93453324, ..., -0.05449919,
         0.06020733, -0.54515416],
       ...,
       [ 0.87686984,  1.06548848, -0.33441002, ..., -0.23293356,
        -0.98558469,  0.32567395],
       [-0.9560041 , -0.93853666,  0.82123474, ...,  0.55838411,
         0.93615545, -0.54515416],
       [-0.9560041 , -0.93853666, -1.53537419, ..., -0.83030083,
        -0.08871747,  0.06442552]])

In [3]:
X.shape

(442, 10)

In [4]:
y

,progression
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0
...,...
437,178.0
438,104.0
439,132.0
440,220.0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

input_dim = X_train.shape[1]
output_dim =1

model = Sequential()
model.add(Dense(50, input_dim=input_dim))
model.add(Dense(output_dim))
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])

model.summary()
model.fit(X_train, y_train, epochs=10)

prediction = model.predict(X_test)

print('RMSE: ', np.sqrt(mse(y_test, prediction)))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                550       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
10/10 [==============================] - 0s 1ms/step - loss: 29262.3555 - mean_squared_error: 29262.3555
Epoch 2/10
10/10 [==============================] - 0s 2ms/step - loss: 29184.3906 - mean_squared_error: 29184.3906
Epoch 3/10
10/10 [==============================] - 0s 1ms/step - loss: 29105.2324 - mean_squared_error: 29105.2324
Epoch 4/10
10/10 [==============================] - 0s 1ms/step - loss: 29027.3340 - mean_squared_error: 29027.3340
Epoch 5/10
10/10 [==============================] - 0s 1

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import Adam

def nn_model(no_neurons, learning_rate, kernel='relu'):
    model = Sequential()
    model.add(Dense(no_neurons, input_dim=X_train.shape[1]))
    model.add(Activation(kernel))
    
    model.add(Dense(no_neurons))
    model.add(Activation(kernel))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(optimizer=Adam(lr=learning_rate), loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model

In [8]:
import warnings
warnings.simplefilter('ignore')

In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'no_neurons': [5,20], 'kernel':['relu', 'linear'], 'learning_rate':[0.0001,0.01], 'epochs':[5,10], 'verbose':[0]}

grid_search = GridSearchCV(KerasClassifier(nn_model), parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train.values.ravel())

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

print('Mean RMSE (+/- standard deviation), for parameters')
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print('%0.3f (+/- %0.03f) for %r'
         %(np.sqrt(-1*mean), np.sqrt(std), params))

Mean RMSE (+/- standard deviation), for parameters
144.586 (+/- 29.347) for {'epochs': 5, 'kernel': 'relu', 'learning_rate': 0.0001, 'no_neurons': 5, 'verbose': 0}
143.129 (+/- 27.839) for {'epochs': 5, 'kernel': 'relu', 'learning_rate': 0.0001, 'no_neurons': 20, 'verbose': 0}
141.396 (+/- 31.294) for {'epochs': 5, 'kernel': 'relu', 'learning_rate': 0.01, 'no_neurons': 5, 'verbose': 0}
141.396 (+/- 31.294) for {'epochs': 5, 'kernel': 'relu', 'learning_rate': 0.01, 'no_neurons': 20, 'verbose': 0}
144.680 (+/- 29.990) for {'epochs': 5, 'kernel': 'linear', 'learning_rate': 0.0001, 'no_neurons': 5, 'verbose': 0}
144.454 (+/- 29.283) for {'epochs': 5, 'kernel': 'linear', 'learning_rate': 0.0001, 'no_neurons': 20, 'verbose': 0}
141.604 (+/- 31.169) for {'epochs': 5, 'kernel': 'linear', 'learning_rate': 0.01, 'no_neurons': 5, 'verbose': 0}
141.396 (+/- 31.294) for {'epochs': 5, 'kernel': 'linear', 'learning_rate': 0.01, 'no_neurons': 20, 'verbose': 0}
143.080 (+/- 28.869) for {'epochs': 10, '